# II.4 PLU and Cholesky factorisations

In this chapter we consider the following factorisations for square invertible  matrices $A$:
1. The _LU factorisation_:
$ A = LU$ where $L$ is lower triangular and $U$ is upper triangular. This is equivalent to Gaussian elimination without pivoting,
so may not exist (e.g. if $A[1,1] = 0$).
1. The _PLU factorisation_:
$
A = P^⊤ LU
$
where $P$ is a permutation matrix, $L$ is lower triangular and $U$ is upper triangular. This is equivalent to Gaussian elimination with pivoting.
It always exists but may in extremely rare cases be unstable. 
2. For a real square _symmetric positive definite_ ($A ∈ ℝ^{n × n}$ such that $A^⊤ = A$ and $𝐱^⊤ A 𝐱 > 0$ for all $𝐱 ∈ ℝ^n$, $𝐱 ≠ 0$) 
matrix the LU decompostion has a special form which is called the _Cholesky factorisation_:
$
A = L L^⊤
$. This provides an algorithmic way to _prove_ that a matrix is symmetric positive definite.
3. We also discuss timing and stability of the different factorisations.

In [1]:
using LinearAlgebra, Plots, BenchmarkTools

## 1. LU Factorisation

Just as Gram–Schmidt can be reinterpreted as a reduced QR factorisation,
Gaussian elimination  can be interpreted as an LU factorisation. Write a matrix
$A ∈ ℂ^{n × n}$ as follows:
$$
A =  \begin{bmatrix} α_1 & 𝐰_1^⊤ \\ 𝐯_1 & A_2 \end{bmatrix}
$$
where $α_1 = a_{11}$, $𝐯_1 = A[2:n, 1]$ and $𝐰_1 = A[1, 2:n]$. Gaussian elimination consists of taking the first row, dividing by
$α$ and subtracting from all other rows. That is equivalent to multiplying by a lower triangular matrix:
$$
\begin{bmatrix}
1 \\
-𝐯_1/α_1 & I \end{bmatrix} A = \begin{bmatrix} α_1 & 𝐰_1^⊤ \\  & K -𝐯_1𝐰_1^⊤ /α_1 \end{bmatrix}
$$
where $A_2 := K -𝐯_1 𝐰_1^⊤/α_1$  happens to be a rank-1 perturbation of $K$. 
Assume $A_2 = L_2 U_2$.
We can write this another way:
$$
A = \underbrace{\begin{bmatrix}
1 \\
𝐯_1/α_1 & I \end{bmatrix}}_{L_1}  \begin{bmatrix} α_1 & 𝐰_1^⊤ \\  & A_2 \end{bmatrix}
= \underbrace{L_1 \begin{bmatrix}
1 \\
 & L_2 \end{bmatrix}}_{L}  \underbrace{\begin{bmatrix} α_1 & 𝐰_1^⊤ \\  & U_2 \end{bmatrix}}_{U}
$$
Note we can multiply through to find
$$
L = \begin{bmatrix}
1 \\
𝐯_1/α_1 & L_2 \end{bmatrix}
$$

This procedure implies an algorithm:

**Algorithm 1 (LU)** Let $A_1 := A$. For $k = 1, …, n-1$ define
$$
\begin{align*}
α_k &:= A_k[1,1] \\
𝐯_k &:= A_k[2:n-k+1,1] \\
𝐰_k &:= A_k[1, 2:n-k+1] \\
A_{k+1} &:= A_k[2:n-k+1,2:n-k+1]-𝐯_k 𝐰_k^⊤/α_k,
\end{align*}
$$
and $α_n := A_n[1,1]$. 
Then for $ℓ_{kj} := 𝐯_j[k-j]/α_j$ and $u_{kj} = 𝐰_k[j-k]$ we have
$$
A = \underbrace{\begin{bmatrix}1 \\ ℓ_{21} & 1 \\
                ℓ_{31} & ℓ_{32} & 1 \\
                ⋮ &  ⋮ & ⋱ & ⋱ \\
                ℓ_{n1} & ℓ_{n2} & ⋯ & ℓ_{n,n-1} & 1
                \end{bmatrix}}_L \underbrace{\begin{bmatrix} α_1 &u_{12} & u_{13} & ⋯ & u_{1n} \\
                                          & α_2 & u_{23} & ⋯ & u_{2n} \\
                                        && ⋱ & ⋱ & ⋮ \\
                                        &&& α_{n-1} & u_{n-1,n} \\
                                        &&&& α_n\end{bmatrix}}_U
$$





**Example 1 (computer)**
We will do a numerical example (by-hand is equivalent to Gaussian elimination).
The first lower triangular matrix is:

In [2]:
n = 4
A = randn(n,n)
A₁= A

α₁,𝐯₁,𝐰₁ = A₁[1,1],A₁[2:n,1],A₁[1,2:n]
L₁ = I + [0; 𝐯₁/α₁] * [1 zeros(1,n-1)] # a rank-1 correction

4×4 Matrix{Float64}:
  1.0        0.0  0.0  0.0
 -3.96853    1.0  0.0  0.0
  0.0555434  0.0  1.0  0.0
  1.51991    0.0  0.0  1.0

Which indeed introduces zeros in the first column:

In [3]:
L₁\A

4×4 Matrix{Float64}:
 -0.70849  -0.70563  -1.51285   0.385756
  0.0      -1.09433  -5.56782   0.272468
  0.0      -1.50983  -1.11524   1.69382
  0.0       1.79023   2.15805  -0.205982

Now we make the next lower triangular operator:

In [4]:
A₂ = A₁[2:n,2:n] - 𝐯₁*𝐰₁'/α₁ # can use ' since its real. Otherwise use transpose(𝐰₁)
α₂,𝐯₂,𝐰₂ = A₂[1,1],A₂[2:n-1,1],A₂[1,2:n-1]
L₂ = I + [0; 𝐯₂/α₂] * [1 zeros(1,n-2)] # a rank-1 correction

3×3 Matrix{Float64}:
  1.0      0.0  0.0
  1.37968  1.0  0.0
 -1.63591  0.0  1.0

Which has introduced zeros in the second column:

In [5]:
# adds a row and column to L₂
L̃₂ = [1          zeros(1,n-1);
      zeros(n-1) L₂]
L̃₂\(L₁\A)

4×4 Matrix{Float64}:
 -0.70849  -0.70563  -1.51285  0.385756
  0.0      -1.09433  -5.56782  0.272468
  0.0       0.0       6.5666   1.3179
  0.0       0.0      -6.95041  0.239751

The last one is:

In [6]:
A₃ = A₂[2:n-1,2:n-1] - 𝐯₂/α₂ * 𝐰₂' # can use ' since its real. Otherwise use transpose(𝐰₁)
α₃,𝐯₃,𝐰₃ = A₃[1,1],A₃[2:n-2,1],A₃[1,2:n-2]
L₃ = I + [0; 𝐯₃/α₃] * [1 zeros(1,n-3)] # a rank-1 correction

2×2 Matrix{Float64}:
  1.0      0.0
 -1.05845  1.0

Giving us

In [7]:
# adds two rows and columns to L̃₃
L̃₃ = [I(2) zeros(2,n-2); zeros(n-2,2) L₃]
U = L̃₃ \ (L̃₂ \(L₁\A))

4×4 Matrix{Float64}:
 -0.70849  -0.70563  -1.51285  0.385756
  0.0      -1.09433  -5.56782  0.272468
  0.0       0.0       6.5666   1.3179
  0.0       0.0       0.0      1.63468

In other words, `L̃₃*L̃₂*L₁*U == A` and hence we have:

In [8]:
L = L₁*L̃₂*L̃₃
L*U ≈ A

true

The entries of `L` are indeed identical to the negative
lower entries of `L₁`, `L₂` and `L₃`:

In [9]:
L = zeros(n,n)
for k = 1:n
    L[k,k] = 1
end
L[2:n,1] = 𝐯₁/α₁
L[3:n,2] = 𝐯₂/α₂
L[4:n,3] = 𝐯₃/α₃
L ≈ L₁*L̃₂*L̃₃

true

**Example 2 (by-hand)**

Consider the matrix
$$
A = \begin{bmatrix} 1 & 1 & 1 \\
                    2 & 4 & 8 \\
                    1 & 4 & 9
                    \end{bmatrix}
$$
We  write
$$
\begin{align*}
A = \underbrace{\begin{bmatrix} 1  \\
                    2 & 1 &  \\
                    1 &  & 1
                    \end{bmatrix}}_{L_1} \begin{bmatrix} 1 & 1 & 1 \\
                    0 & 2 & 6 \\
                    0 & 3 & 8
                    \end{bmatrix} 
 = L_1 \underbrace{\begin{bmatrix} 1  \\
                     & 1 &  \\
                     & 3/2 & 1
                    \end{bmatrix}}_{L̃_2} \begin{bmatrix} 1 & 1 & 1 \\
                    0 & 2 & 6 \\
                    0 & 0 & 5
                    \end{bmatrix} \\
= \underbrace{\begin{bmatrix} 1  \\
                    2 & 1 &  \\
                    1 & 3/2 & 1
                    \end{bmatrix}}_{L} \underbrace{\begin{bmatrix} 1 & 1 & 1 \\
                    0 & 2 & 6 \\
                    0 & 0 & 5
                    \end{bmatrix}}_U
\end{align*}
$$




## 2. PLU Factorisation

We learned in first year linear algebra that if a diagonal entry is zero
whem doing Gaussian elimnation one has to _row pivot_. For stability, 
in implementation one _always_ pivots: swap the largest in magnitude entry for the entry on the diagonal.
We will see this is equivalent to a PLU decomposition:


**Theorem 1 (PLU)** A matrix $A ∈ ℂ^{n × n}$ is invertible if and only if it has a PLU decomposition:
$$
A = P^⊤ L U
$$
where the diagonal of $L$ are all equal to 1 and the diagonal of $U$ are all non-zero.

**Proof**

If we have a PLU decomposition of this form then $L$ and $U$ are invertible and hence the inverse is simply $A^{-1} = U^{-1} L^{-1} P$. 

We will prove the other direction by induction.
If $A ∈ ℂ^{1 × 1}$ we trivially have an LU decomposition $A = [1] * [a_{11}]$ as all matrices are triangular.
As $A$ is invertible not all entries in the first column are zero. Therefore there exists a permutation
$P_1$ so that $α := (P_1 A)[1,1] ≠ 0$. Hence we write
$$
P_1 A = \begin{bmatrix} α & 𝐰^⊤ \\
                        𝐯 & A_2
                        \end{bmatrix} = \underbrace{\begin{bmatrix}
1 \\
𝐯/α & I \end{bmatrix}}_{L_1}  \begin{bmatrix} α & 𝐰^⊤ \\  & A_2 \end{bmatrix}
$$
We know $A_2$ is invertible: if $A_2^⊤ 𝐜  = 𝟎$ then for $𝐝 := P_1^⊤ L_1^{-⊤} [0; 𝐜]$ we have
$$
\begin{align*}
𝐝^⊤ A =  [0\ 𝐜^⊤] L_1^{-1} P_1 P_1^⊤ L_1  \begin{bmatrix} α & 𝐰^⊤ \\  & A_2 \end{bmatrix}
= [0\ 𝐜^⊤] \begin{bmatrix} α & 𝐰^⊤ \\  & A_2 \end{bmatrix} = [0\ 𝐜^⊤\ A_2] = 𝟎^⊤.
\end{align*}
$$
Thus by induction $A_2 = P_2^⊤ L_2 U_2$. Thus we have:
$$
\begin{align*}
\begin{bmatrix} 1 \\
            & P_2 \end{bmatrix} P_1 A &= \begin{bmatrix} 1 \\
            & P_2 \end{bmatrix}  \begin{bmatrix} α & 𝐰^⊤ \\
                        𝐯 & A_2
                        \end{bmatrix}  =
            \begin{bmatrix} 1 \\ & P_2 \end{bmatrix} L_1  \begin{bmatrix} α & 𝐰^⊤ \\  & P_2^⊤ L_2 U_2 \end{bmatrix} \\
            &= \begin{bmatrix}
1 \\
P_2 𝐯/α & P_2 \end{bmatrix} \begin{bmatrix} 1 &  \\  &  P_2^⊤ L_2 \end{bmatrix}  \begin{bmatrix} α & 𝐰^⊤ \\  &  U_2 \end{bmatrix} \\
&= \underbrace{\begin{bmatrix}
1 \\
P_2 𝐯/α & L_2 \end{bmatrix}}_L \underbrace{\begin{bmatrix} α & 𝐰^⊤ \\  &  U_2 \end{bmatrix}}_U \\
\end{align*}
$$

∎

In the above we neglected to state which permutation is used as for the proof of existence it is immaterial. For _stability_ however, 
we choose one that puts the largest entry: let $σ_{\rm max} : ℝ^n → S_n$ be the permutation that swaps the first row with the row
of $𝐚$ whose absolute value is maximised.
In cycle notation we then have:
$$
σ_{\rm max}(𝐚) = (1, {\rm indmax} |𝐚|)
$$
where ${\rm indmax}$ gives the index of the entry of a vector which is its maximum.
This inductive proof encodes an  algorithm:

**Algorithm 2 (PLU)** Let $A_1 := A$. For $k = 1, …, n-1$ define
$$
\begin{align*}
σ_k &:= σ_{\rm max}(A_k[1:n-k+1,1]) \\
P_k &:= P_{σ_k} \\
Ã_k & := P_k A_k \\
α_k &:= Ã_k[1,1] \\
𝐯_k &:= Ã_k[2:n-k+1,1] \\
𝐰_k &:= Ã_k[1, 2:n-k+1] \\
A_{k+1} &:= Ã_k[2:n-k+1,2:n-k+1]-𝐯_k 𝐰_k^⊤/α_k,
\end{align*}
$$
and finally $α_n := A_n[1,1]$. 
Then for $ℓ_{kj} := 𝐯_j[k-j]/α_j$, $u_{kj} = 𝐰_k[j-k]$, and
$$
P̃_k := \begin{bmatrix} I_{k-1} \\ & P_k \end{bmatrix}
$$
 we have
$$
\underbrace{P̃_{σ_{n-1}} ⋯ P̃_{σ_1}}_P A = \underbrace{\begin{bmatrix}1 \\ ℓ_{21} & 1 \\
                ℓ_{31} & ℓ_{32} & 1 \\
                ⋮ &  ⋮ & ⋱ & ⋱ \\
                ℓ_{n1} & ℓ_{n2} & ⋯ & ℓ_{n,n-1} 1
                \end{bmatrix}}_L \underbrace{\begin{bmatrix} α_1 &u_{12} & u_{13} & ⋯ & u_{1n} \\
                                          & α_2 & u_{23} & ⋯ & u_{2n} \\
                                        && ⋱ & ⋱ & ⋮ \\
                                        &&& α_{n-1} & u_{n-1,n} \\
                                        &&&& α_n\end{bmatrix}}_U
$$


**Example 3**

Again we consider the matrix
$$
A = \begin{bmatrix} 1 & 1 & 1 \\
                    2 & 4 & 8 \\
                    1 & 4 & 9
                    \end{bmatrix}
$$
Even though $a_{11} = 1 ≠ 0$, we still pivot: placing 
the maximum entry on the diagonal to mitigate numerical errors.
That is, we first pivot and upper triangularise the first column:
$$
 L_1 P_1 A =  L_1\begin{bmatrix} 0 & 1 \\ 1 & 0 \\ && 1 \end{bmatrix}
\begin{bmatrix} 1 & 1 & 1 \\
                    2 & 4 & 8 \\
                    1 & 4 & 9
                    \end{bmatrix} = 
                     \begin{bmatrix}1 \\ -{1 \over 2} & 1 \\ -{1 \over 2} && 1 \end{bmatrix}
\begin{bmatrix} 2 & 4 & 8 \\
                1 & 1 & 1 \\
                    1 & 4 & 9
                    \end{bmatrix}
$$
We now pivot and upper triangularise the second column:
$$
  L̃_2 P̃_2 L̃_1 P_1 A = 
                    L̃_2 \begin{bmatrix}
                    1 \\ &0 & 1 \\ &1 & 0 \end{bmatrix}
\begin{bmatrix} 2 & 4 & 8 \\
                0 & -1 & -3 \\
                    0 & 2 & 5
                    \end{bmatrix}
                    = \begin{bmatrix} 1\\ & 1 \\ & {1 \over 2} & 1 \end{bmatrix}
\begin{bmatrix} 2 & 4 & 8 \\
                0 & 2 & 5 \\
                0 & -1 & -3 
                    \end{bmatrix} = 
                    \underbrace{\begin{bmatrix} 2 & 4 & 8 \\
                0 & 2 & 5 \\
                0 & 0 & -{1 \over 2}
                    \end{bmatrix}}_U
$$
We now move $P̃_2$ to the right:
$$
L_2 P_2 L_1 P_1 = \underbrace{\begin{bmatrix} 1\\ -{1 \over 2} & 1 \\  -{1 \over 2}  & {1 \over 2} & 1 \end{bmatrix}}_{L_2 \tilde L_1} \underbrace{\begin{bmatrix} 0 & 1 & 0 \\ 0 & 0 & 1 \\ 1 & 0 & 0 \end{bmatrix}}_P
$$
That is
$$
L = \begin{bmatrix} 1\\ {1 \over 2} & 1 \\  {1 \over 2}  & -{1 \over 2} & 1 \end{bmatrix}
$$

We see how this example is done on a computer:

In [10]:
A = [1 1 1;
     2 4 8;
     1 4 9]
L,U,σ = lu(A) # σ is a vector encoding the permutation

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0   0.0  0.0
 0.5   1.0  0.0
 0.5  -0.5  1.0
U factor:
3×3 Matrix{Float64}:
 2.0  4.0   8.0
 0.0  2.0   5.0
 0.0  0.0  -0.5

The permutation is

In [11]:
σ

3-element Vector{Int64}:
 2
 3
 1

Thus to invert a system we can do:

In [12]:
b = randn(3)
U\(L\b[σ]) == A\b

true

Note the entries match exactly because this precisely what `\` is using.

## 3. Cholesky Factorisation

Cholesky Factorisation is a form of Gaussian elimination (without pivoting)
that exploits symmetry in the problem, resulting in a substantial speedup. 
It is only relevant for _symmetric positive definite_
matrices.

**Definition 1 (positive definite)** A square matrix $A ∈ ℝ^{n × n}$ is _positive definite_ if
for all $𝐱 ∈ ℝ^n, x ≠ 0$ we have
$$
𝐱^⊤ A 𝐱 > 0
$$

First we establish some basic properties of positive definite matrices:

**Proposition 3 (conj. pos. def.)** If  $A ∈ ℝ^{n × n}$ is positive definite and 
$V ∈ ℝ^{n × n}$ is non-singular then
$$
V^⊤ A V
$$
is positive definite.

**Proposition 4 (diag positivity)** If $A ∈ ℝ^{n × n}$ is positive definite
then its diagonal entries are positive: $a_{kk} > 0$.


**Theorem 1 (subslice pos. def.)** If $A ∈ ℝ^{n × n}$ is positive definite
and $𝐤 ∈ \{1,\ldots,n\}^m$ is a vector of $m$ integers where any integer appears only once,
 then $A[𝐤,𝐤] ∈ ℝ^{m × m}$ is also
positive definite.



We leave the proofs to the problem sheets. Here is the key result:


**Theorem 2 (Cholesky and sym. pos. def.)** A matrix $A$ is symmetric positive definite if and only if it has a Cholesky factorisation
$$
A = L L^⊤
$$
where the diagonals of $L$ are positive.

**Proof** If $A$ has a Cholesky factorisation it is symmetric ($A^⊤ = (L L^⊤)^⊤ = A$) and for $𝐱 ≠ 0$ we have
$$
𝐱^⊤ A 𝐱 = (L𝐱)^⊤ L 𝐱 = \|L𝐱\|^2 > 0
$$
where we use the fact that $L$ is non-singular.

For the other direction we will prove it by induction, with the $1 × 1$ case being trivial. 
Assume all lower dimensional symmetric positive definite matrices have Cholesky decompositions.
Write
$$
A = \begin{bmatrix} α & 𝐯^⊤ \\
                    𝐯   & K
                    \end{bmatrix} = \underbrace{\begin{bmatrix} \sqrt{α} \\ 
                                    {𝐯 \over \sqrt{α}} & I \end{bmatrix}}_{L_1}
                                    \underbrace{\begin{bmatrix} 1  \\ & K - {𝐯 𝐯^⊤ \over α} \end{bmatrix}}_{A_1}
                                    \underbrace{\begin{bmatrix} \sqrt{α} & {𝐯^⊤ \over \sqrt{α}} \\
                                     & I \end{bmatrix}}_{L_1^⊤}.
$$
Note that $A_2 := K - {𝐯 𝐯^⊤ \over α}$ is a subslice of $L_1^{-1} A L_1^{-⊤}$, hence by the previous propositions is
itself symmetric positive definite. Thus we can write 
$$
A_2 = K - {𝐯 𝐯^⊤ \over α} = L_2 L_2^⊤
$$
and hence $A = L L^⊤$ for
$$
L= L_1 \begin{bmatrix}1 \\ & L_2 \end{bmatrix} = \begin{bmatrix} \sqrt{α} \\ {𝐯 \over \sqrt{α}} & L_2 \end{bmatrix}
$$
satisfies $A = L L^⊤$.
∎


Note hidden in this proof is a simple algorithm form computing the Cholesky factorisation.


**Algorithm (Cholesky)** Let $A_1 := A$. For  $k = 1, …, n$ define
$$
\begin{align*}
α_k &:= A_k[1,1] \\
𝐯_k &:= A_k[2:n-k+1,1] \\
A_{k+1} &:= A_k[2:n-k+1,2:n-k+1] - {𝐯_k 𝐯_k^⊤ \over α_k}.
\end{align*}
$$
Then
$$
L = \begin{bmatrix} \sqrt{α_1} \\
    {𝐯_1[1] \over \sqrt{α_1}}  &  \sqrt{α_2} \\
    {𝐯_1[2] \over \sqrt{α_1}}  & {𝐯_2[1] \over \sqrt{α_2}} &  \sqrt{α_2} \\
    ⋮ & ⋮ & ⋱ & ⋱ \\
    {𝐯_1[n-1] \over \sqrt{α_1}} &{𝐯_2[n-2] \over \sqrt{α_2}} & ⋯ & {𝐯_{n-1}[1] \over \sqrt{α_{n-1}}} & \sqrt{α_{n}}
    \end{bmatrix}
$$

This algorithm succeeds if and only if $A$ is symmetric positive definite.

**Example 4** Consider the matrix
$$
A_0 = A = \begin{bmatrix}
2 &1 &1 &1 \\
1 & 2 & 1 & 1 \\
1 & 1 & 2 & 1 \\
1 & 1 & 1 & 2
\end{bmatrix}
$$
Then
$$
A_1 = \begin{bmatrix}
2 &1 &1 \\
1 & 2 & 1 \\
1 & 1 & 2 
\end{bmatrix} - {1 \over 2} \begin{bmatrix} 1 \\ 1 \\ 1 \end{bmatrix} \begin{bmatrix} 1 & 1 & 1 \end{bmatrix} =
{1 \over 2} \begin{bmatrix}
3 & 1 & 1 \\
1 & 3 & 1 \\
1 & 1 & 3 
\end{bmatrix}
$$
Continuing, we have 
$$
A_2 = {1 \over 2} \left( \begin{bmatrix}
3 & 1 \\ 1 & 3
\end{bmatrix} - {1 \over 3} \begin{bmatrix} 1 \\ 1  \end{bmatrix} \begin{bmatrix} 1 & 1  \end{bmatrix}
\right)
= {1 \over 3} \begin{bmatrix} 4 & 1 \\ 1 & 4 \end{bmatrix}
$$
Finally
$$
A_3 = {5 \over 4}.
$$
Thus we get
$$
L= L_1 L_2 L_3 = \begin{bmatrix} \sqrt{2} \\ {1 \over \sqrt{2}} & {\sqrt{3} \over 2} \\ 
{1 \over \sqrt{2}} & {1 \over \sqrt 6} & {2 \over \sqrt{3}} \\
{1 \over \sqrt{2}} & {1 \over \sqrt 6} & {1 \over \sqrt{12}} & {\sqrt{5} \over 2}
\end{bmatrix}
$$


# 3. Timings and Stability

The different factorisations have trade-offs between speed and stability.
First we compare the speed of the different factorisations on a symmetric positive
definite matrix, from fastest to slowest:

In [13]:
n = 100
A = Symmetric(rand(n,n)) + 100I # shift by 10 ensures positivity
@btime cholesky(A);
@btime lu(A);
@btime qr(A);

  303.311 μs (3 allocations: 78.20 KiB)
  79.607 μs (4 allocations: 79.08 KiB)
  256.592 μs (7 allocations: 134.55 KiB)


On my machine, `cholesky` is ~1.5x faster than `lu`,  
which is ~2x faster than QR. 



In terms of stability, QR computed with Householder reflections
(and Cholesky for positive definite matrices) are stable, 
whereas LU is usually unstable (unless the matrix
is diagonally dominant). PLU is a very complicated story: in theory it is unstable,
but the set of matrices for which it is unstable is extremely small, so small one does not
normally run into them.

Here is an example matrix that is in this set.

In [14]:
function badmatrix(n)
    A = Matrix(1I, n, n)
    A[:,end] .= 1
    for j = 1:n-1
        A[j+1:end,j] .= -1
    end
    A
end
A =1badmatrix(5)

5×5 Matrix{Int64}:
  1   0   0   0  1
 -1   1   0   0  1
 -1  -1   1   0  1
 -1  -1  -1   1  1
 -1  -1  -1  -1  1

Note that pivoting will not occur (we do not pivot as the entries below the diagonal are the same magnitude as the diagonal), thus the PLU Factorisation is equivalent to an LU factorisation:

In [15]:
L,U = lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
5×5 Matrix{Float64}:
  1.0   0.0   0.0   0.0  0.0
 -1.0   1.0   0.0   0.0  0.0
 -1.0  -1.0   1.0   0.0  0.0
 -1.0  -1.0  -1.0   1.0  0.0
 -1.0  -1.0  -1.0  -1.0  1.0
U factor:
5×5 Matrix{Float64}:
 1.0  0.0  0.0  0.0   1.0
 0.0  1.0  0.0  0.0   2.0
 0.0  0.0  1.0  0.0   4.0
 0.0  0.0  0.0  1.0   8.0
 0.0  0.0  0.0  0.0  16.0

But here we see an issue: the last column of `U` is growing exponentially fast! Thus when `n` is large
we get very large errors:

In [16]:
n = 100
b = randn(n)
A = badmatrix(n)
norm(A\b - qr(A)\b) # A \ b still uses lu

4.03401008242218

Note `qr` is completely fine:

In [17]:
norm(qr(A)\b - qr(big.(A)) \b) # roughly machine precision

6.224059546039034056825914166660385772183212869564350231828315869079770957991858e-15

Amazingly, PLU is fine if applied to a small perturbation of `A`:

In [18]:
ε = 0.000001
Aε = A .+ ε .* randn.()
norm(Aε \ b - qr(Aε) \ b) # Now it matches!

8.747274731765869e-15

The big _open problem_ in numerical linear algebra is to prove that the set of matrices
for which PLU fails has extremely small measure.